# Benchmarking tflite models  

This notebook parses the benchmarking.txt files generated by: 

 [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)  

 TensorFlow Lite benchmark tools currently measure and calculate statistics for the following important performance metrics:

- Initialization time
- Inference time of warmup state
- Inference time of steady state
- Memory usage during initialization time
- Overall memory usage

In [2]:
import os, sys, math, datetime
import pathlib
from pathlib import Path

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths
from workbench.tflite_benchmarking import get_index_of_model_profiling_lines
from workbench.wandb import wandb_model_DB, make_clickable, get_architecture_from_model_DB_run_id, get_model_DB_run_id_from_architecture
from workbench.tflite_profiling import get_peak_memory_df, get_tensor_details_df

%load_ext autoreload

In [3]:
import re
from matplotlib import pyplot as plt
#import plotly.express as px
import pandas as pd


# enable plotly in VS Studio Code
#import plotly.io as pio
#pio.renderers.default = "notebook_connected"
#pio.renderers.default = "plotly_mimetype+notebook"

import wandb

In [4]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [5]:
models_dir = initialize()

In [6]:
automated = False
id = wandb.sdk.lib.runid.generate_id() 

global model_name

model_name = "mobilenetv2_0.25_96_c3_o2_t5l512.MV1"


models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

/mnt/c/tiny_mlc/tiny_cnn/models


In [7]:
models_benchmark_path = models_dir.joinpath(model_name, f"{model_name}_benchmark.txt")
models_benchmark_path

PosixPath('/mnt/c/tiny_mlc/tiny_cnn/models/mobilenetv2_0.25_96_c3_o2_t5l512.MV1/mobilenetv2_0.25_96_c3_o2_t5l512.MV1_benchmark.txt')

In [8]:
#! explorer $models_benchmark_path

# Helper functions

In [9]:
def clean_model_summary(filepath): 

    if not filepath.is_file():
        print(f"This file does not exist: {filepath}")
        return None

    else:
        clean_lines = []
        # Parse the MLTK model summary to grab important metrics   
        with open(filepath, "r", encoding="latin-1") as f:
            lines = f.readlines() # list containing lines of file
            for line in lines:
                line = line.strip() # remove leading/trailing white spaces
                if len(line)> 0:
                    
                    clean_lines.append(line)
                else:
                    pass
            #columns = [] # To store column names
        return clean_lines

In [10]:
def clean_column_names(df):
    cols = df.columns

    clean_cols = []
    for col in cols:
        col = col.strip()
        col = col.replace("[" , "")
        col = col.replace("]" , "")   
        clean_cols.append(col)
        
    return clean_cols

In [11]:
def string_percent_to_float(x):
    x = str(x).strip("%")
    return float(x)

In [12]:
def remove_tabs(text):
    """removes tabs from a list of strings

    Args:
        text (list(str)): list of strings that contains tabs

    Returns:
        list(str): list of strings without tabs
    """
    split_text= []
    for l in text:
        split_text.append((l.split("\t")))
        
    return split_text

# Parsing the model file

if models_benchmark_path.is_file():
    

In [13]:
lines = clean_model_summary(models_benchmark_path)
lines

['STARTING!',
 'Log parameter values verbosely: [0]',
 'Num threads: [1]',
 'Graph: [/mnt/c/tiny_mlc/tiny_cnn/models/mobilenetv2_0.25_96_c3_o2_t5l512.MV1/mobilenetv2_0.25_96_c3_o2_t5l512.MV1_INT8.tflite]',
 'Enable op profiling: [1]',
 '#threads used for CPU inference: [1]',
 'Loaded model /mnt/c/tiny_mlc/tiny_cnn/models/mobilenetv2_0.25_96_c3_o2_t5l512.MV1/mobilenetv2_0.25_96_c3_o2_t5l512.MV1_INT8.tflite',
 'The input model file size (MB): 0.282144',
 'Initialized session in 120.179ms.',
 'Running benchmark for at least 1 iterations and at least 0.5 seconds but terminate if exceeding 150 seconds.',
 'count=736 first=3866 curr=782 min=435 max=4658 avg=676.192 std=332',
 'Running benchmark for at least 50 iterations and at least 1 seconds but terminate if exceeding 150 seconds.',
 'count=1750 first=851 curr=652 min=452 max=1421 avg=536.384 std=133',
 'Inference timings in us: Init: 120179, First inference: 3866, Warmup (avg): 676.192, Inference (avg): 536.384',
 'Note: as the benchmark 

In [14]:
def get_profiling_lines(lines):
    #print(lines)
    # find section split
    for i, line in enumerate(lines):
        #print(f"{i} : {line}")
        #if line.contains('Operator-wise'):
        if ("Operator-wise" in line):
            split_line = i
            print(f"{i} - profiling")
            #print(f"Splitting text at line {split_line}")
            model_profiling = lines[:split_line]
            operator_profiling = lines[split_line+1:]
            return model_profiling, operator_profiling
        else:
            pass
    print("WARNING: Could not find the text: 'Operator-wise Profiling Info for Regular Benchmark Runs:'")
    return (None, None)

In [15]:
model_profiling, operator_profiling = get_profiling_lines(lines)

33 - profiling


In [16]:
model_lines_dict = get_index_of_model_profiling_lines(model_profiling)

In [17]:
def get_model_profiling_stats(model_profiling):
    model_lines_dict = get_index_of_model_profiling_lines(model_profiling)

    model_stats = {}

    # extract model size
    model_size_string = model_profiling[model_lines_dict["model_file_size"]]
    model_size_string
    model_size= float(model_size_string.split(":")[-1].strip())
    model_stats["model_size_MB"] = model_size

    # extract inference times
    inference_string = model_profiling[model_lines_dict["inference_timings"]]
    #print(inference_string)
    inference_strings = inference_string.split(",")
    for item in inference_strings:
        item_time = item.split()[-1]
        #print(item_time)

    model_stats["init_us"] = int(inference_strings[0].split()[-1].strip())
    model_stats["first_inference_us"] = int(inference_strings[1].split()[-1].strip())
    model_stats["warmup_avg_us"] = float(inference_strings[2].split()[-1].strip())
    model_stats["inference_avg_us"] = float(inference_strings[3].split()[-1].strip())

    # model initialization timings
    init_string = model_profiling[model_lines_dict["initialization_ms"]]
    model_stats["initialization_ms"] = float(init_string.split()[-1].replace("ms.",""))


    # initialization run order
    init_run_order = model_profiling[model_lines_dict["run_order"]: model_lines_dict["top_by_computation_time"]]
    init_run_order = remove_tabs(init_run_order)

    model_stats["modify_graph_with_delegate_ms_first"] = float(init_run_order[2][1].strip())
    model_stats["modify_graph_with_delegate_ms_avg"] = float(init_run_order[2][2].strip())
    model_stats["modify_graph_with_delegate_ms_%"] = float(init_run_order[2][3].strip().replace("%", ""))
    model_stats["modify_graph_with_delegate_mem_KB"] = float(init_run_order[2][5].strip())

    model_stats["allocate_tensors_ms_first"] = float(init_run_order[3][1].strip())
    model_stats["allocate_tensors_ms_avg"] = float(init_run_order[3][2].strip())
    model_stats["allocate_tensors_ms_%"] = float(init_run_order[3][3].strip().replace("%", ""))

    return model_stats

In [18]:
model_stats = get_model_profiling_stats(model_profiling)
model_stats

{'model_size_MB': 0.282144,
 'init_us': 120179,
 'first_inference_us': 3866,
 'warmup_avg_us': 676.192,
 'inference_avg_us': 536.384,
 'initialization_ms': 120.179,
 'modify_graph_with_delegate_ms_first': 7.477,
 'modify_graph_with_delegate_ms_avg': 7.477,
 'modify_graph_with_delegate_ms_%': 99.455,
 'modify_graph_with_delegate_mem_KB': 1132.0,
 'allocate_tensors_ms_first': 0.041,
 'allocate_tensors_ms_avg': 0.041,
 'allocate_tensors_ms_%': 0.545}

In [19]:
def get_profiling_stats_cpu(filepath):
    """
    Parse "bechmarking.txt" file that has been generated from a .tflite model with TensorFlow benchmarking tool.  

    The native benchmark binary for linux is available from this page:  
    [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)   

    Args:
        filepath (pathlib.Path): filepath to the extracted .txt file

    Returns:
        dict: dictionary with extracted model summary profiling information 
    """

    lines = clean_model_summary(filepath)
    model_profiling, operator_profiling = get_profiling_lines(lines)
    profiling_stats = get_model_profiling_stats(model_profiling)
    return profiling_stats


In [20]:
prof_stats = get_profiling_stats_cpu(models_benchmark_path)
prof_stats

33 - profiling


{'model_size_MB': 0.282144,
 'init_us': 120179,
 'first_inference_us': 3866,
 'warmup_avg_us': 676.192,
 'inference_avg_us': 536.384,
 'initialization_ms': 120.179,
 'modify_graph_with_delegate_ms_first': 7.477,
 'modify_graph_with_delegate_ms_avg': 7.477,
 'modify_graph_with_delegate_ms_%': 99.455,
 'modify_graph_with_delegate_mem_KB': 1132.0,
 'allocate_tensors_ms_first': 0.041,
 'allocate_tensors_ms_avg': 0.041,
 'allocate_tensors_ms_%': 0.545}

# Operator profiling

In [63]:
def get_operator_df(text, name=""):
    df = pd.DataFrame(text)
    df.rename(columns=df.iloc[0, :], inplace=True) 
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)
    # try:
    df["%"] = df["%"].apply(string_percent_to_float)
    # except:
    #     df["avg %"] = df["avg %"].apply(string_percent_to_float)
    # try:
    df["cdf%"] = df["cdf%"].apply(string_percent_to_float)
    # except:
    #     df["cdf %"] = df["cdf %"].apply(string_percent_to_float)
    df["first"] = df["first"].map(float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)#
    df.reset_index(inplace=True, drop=False)
    df.rename({"index": "_index"}, inplace=True)
    #df.Name = name

    return df

In [64]:
def get_node_df(text, name=""):
    df = pd.DataFrame(text)
    df.rename(columns=df.iloc[0, :], inplace=True) 
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)
    df["avg %"] = df["avg %"].apply(string_percent_to_float)
    df["cdf %"] = df["cdf %"].apply(string_percent_to_float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)
    df["count"] = df["count"].map(int)
    df.Name = name

    return df

In [65]:
def get_index_of_operator_profiling_lines(lines):
    operator_lines_dict = {}
    for i, line in enumerate(lines):
        #if line.str.contains('= Run Order ='):
        if "= Run Order =" in line:
            operator_lines_dict["run_order"] = i
        elif "= Top by Computation Time =" in line:
            operator_lines_dict["top_by_computation_time"] = i
        elif "= Summary by node type =" in line:
            operator_lines_dict["summary_by_node_type"] = i

    # print(operator_lines_dict)
    return operator_lines_dict

In [66]:
# operator_lines_dict = get_index_of_operator_profiling_lines(operator_profiling)
# operator_lines_dict

In [67]:
def get_summary_by_node_type_df(operator_profiling, operator_lines_dict, model_name):
    summary_by_node_type = operator_profiling[operator_lines_dict["summary_by_node_type"] +1:-3]
    summary_by_node_type = remove_tabs(summary_by_node_type)
    summary_by_node_type_df = get_node_df(summary_by_node_type, name=f"Summary by node type - {model_name}")
    return summary_by_node_type_df

In [68]:
def get_operator_top_by_comp_time_df(operator_profiling, operator_lines_dict, model_name):

    top_by_comp_time = operator_profiling[
        operator_lines_dict["top_by_computation_time"]+ 1 : operator_lines_dict["summary_by_node_type"]- 1]
    top_by_comp_time_clean = remove_tabs(top_by_comp_time)

    df = pd.DataFrame(top_by_comp_time_clean)

    df.rename(columns=df.iloc[0, :], inplace=True)
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)

    df["%"] = df["%"].apply(string_percent_to_float)
    df["cdf%"] = df["cdf%"].apply(string_percent_to_float)
    df["first"] = df["first"].map(float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)

    return df

In [69]:
def get_run_order_df(operator_lines_dict, operator_profiling, model_name):
    run_order = operator_profiling[operator_lines_dict["run_order"] +1: operator_lines_dict["top_by_computation_time"]]
    run_order = remove_tabs(run_order)
    df = get_operator_df(run_order, name= f"Run order - {model_name}")
    return df

In [70]:
def get_profiling_dataframes_cpu(filepath, model_name):
    """
    Parse "bechmarking.txt" file that has been generated from a .tflite model with TensorFlow benchmarking tool.  

    The native benchmark binary for linux is available from this page:  
    [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)   

    Args:
        filepath (pathlib.Path): filepath to the extracted .txt file

    Returns:
        list(pd.DataFrame, pd.DataFrame, pd.DataFrame): tuple containing the following dataframes:

            operator_run_order_df, operator_by_comp_time_df, summary_node_type_df
    """

    lines = clean_model_summary(filepath)
    model_profiling, operator_profiling = get_profiling_lines(lines)
    operator_lines_dict = get_index_of_operator_profiling_lines(operator_profiling)

    operator_run_order_df = get_run_order_df(operator_lines_dict, operator_profiling, model_name)
    operator_by_comp_time_df = get_operator_top_by_comp_time_df(operator_profiling, operator_lines_dict, model_name)
    summary_node_type_df = get_summary_by_node_type_df(operator_profiling, operator_lines_dict, model_name)

    return operator_run_order_df, operator_by_comp_time_df, summary_node_type_df

In [71]:
operator_run_order_df, operator_by_comp_time_df, summary_node_type_df = get_profiling_dataframes_cpu(models_benchmark_path, model_name)

33 - profiling


In [72]:
operator_run_order_df

,index,node type,first,avg ms,%,cdf%,mem KB,times called,Name
0,1,QUANTIZE,0.189,0.107,22.194,22.194,0.0,1,[tfl.quantize]:0
1,2,"Convolution (NHWC, QC8) IGEMM",0.071,0.048,9.950,32.144,0.0,1,"Delegate/Convolution (NHWC, QC8) IGEMM:0"
2,3,"Convolution (NHWC, QC8) DWConv",0.097,0.037,7.747,39.891,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:1"
3,4,"Convolution (NHWC, QC8) GEMM",0.025,0.017,3.582,43.472,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:2"
4,5,"Convolution (NHWC, QC8) GEMM",0.052,0.039,7.980,51.452,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:3"
...,...,...,...,...,...,...,...,...,...
63,64,AVERAGE_POOL_2D,0.006,0.004,0.760,99.514,0.0,1,[mobilenetv2/average_pooling2d/AvgPool]:63
64,65,RESHAPE,0.002,0.001,0.120,99.634,0.0,1,[mobilenetv2/flatten/Reshape]:64
65,66,"Fully Connected (NC, QS8) GEMM",0.001,0.000,0.035,99.668,0.0,1,"Delegate/Fully Connected (NC, QS8) GEMM:0"
66,67,SOFTMAX,0.002,0.001,0.165,99.834,0.0,1,[StatefulPartitionedCall:01]:66


In [73]:
operator_by_comp_time_df

,node type,first,avg ms,%,cdf%,mem KB,times called,Name
1,QUANTIZE,0.189,0.107,22.194,22.194,0.0,1,[tfl.quantize]:0
2,"Convolution (NHWC, QC8) IGEMM",0.071,0.048,9.950,32.144,0.0,1,"Delegate/Convolution (NHWC, QC8) IGEMM:0"
3,"Convolution (NHWC, QC8) GEMM",0.052,0.039,7.980,40.124,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:3"
4,"Convolution (NHWC, QC8) DWConv",0.097,0.037,7.747,47.871,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:1"
5,"Convolution (NHWC, QC8) DWConv",0.039,0.027,5.586,53.457,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:7"
6,"Convolution (NHWC, QC8) DWConv",0.031,0.022,4.562,58.019,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:4"
7,"Convolution (NHWC, QC8) GEMM",0.025,0.017,3.582,61.600,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:2"
8,"Convolution (NHWC, QC8) GEMM",0.026,0.013,2.668,64.268,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:10"
9,"Convolution (NHWC, QC8) GEMM",0.017,0.013,2.613,66.881,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:6"
10,"Convolution (NHWC, QC8) GEMM",0.014,0.011,2.193,69.074,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:61"


In [74]:
summary_node_type_df

,Node type,count,avg ms,avg %,cdf %,mem KB,times called
1,"Convolution (NHWC, QC8) GEMM",34,0.156,34.061,34.061,0.0,34
2,"Convolution (NHWC, QC8) DWConv",17,0.143,31.223,65.284,0.0,17
3,QUANTIZE,2,0.107,23.362,88.646,0.0,2
4,"Convolution (NHWC, QC8) IGEMM",1,0.048,10.480,99.127,0.0,1
5,AVERAGE_POOL_2D,1,0.003,0.655,99.782,0.0,1
6,"Add (ND, QS8)",10,0.001,0.218,100.000,0.0,10
7,SOFTMAX,1,0.000,0.000,100.000,0.0,1
8,RESHAPE,1,0.000,0.000,100.000,0.0,1
9,"Fully Connected (NC, QS8) GEMM",1,0.000,0.000,100.000,0.0,1


In [75]:
# operator_profiling = lines[split_line+1:]
# operator_profiling

In [76]:
# operator_lines_dict = {}
# for i, line in enumerate(operator_profiling):
#     #if line.str.contains('= Run Order ='):
#     if "= Run Order =" in line:
#         operator_lines_dict["run_order"] = i
#     elif "= Top by Computation Time =" in line:
#         operator_lines_dict["top_by_computation_time"] = i
#     elif "= Summary by node type =" in line:
#         operator_lines_dict["summary_by_node_type"] = i

# operator_lines_dict

In [77]:
# summary_by_node_type = operator_profiling[operator_lines_dict["summary_by_node_type"] +1:-3]
# summary_by_node_type = remove_tabs(summary_by_node_type)
# summary_by_node_type

In [78]:
# summary_by_node_type_df = get_node_df(summary_by_node_type, name=f"Summary by node type - {model_name}")
# summary_by_node_type_df

In [79]:
#summary_by_node_type_df.Name

In [80]:
#summary_by_node_type_df.info

In [81]:
# top_by_comp_time = operator_profiling[operator_lines_dict["top_by_computation_time"] +1: operator_lines_dict["summary_by_node_type"]-1]
# top_by_comp_time = remove_tabs(top_by_comp_time)
# top_by_comp_time

In [82]:
# df_operator_top_by_comp_time = get_operator_df(top_by_comp_time, name=f"Top by compute time - {model_name}")
# df_operator_top_by_comp_time

In [83]:
# run_order = operator_profiling[operator_lines_dict["run_order"] +1: operator_lines_dict["top_by_computation_time"]]
# run_order = remove_tabs(run_order)
# #run_order

In [84]:
# df_operator_run_order = get_operator_df(run_order, name= f"Run order - {model_name}")
# #df_operator_run_order.style.set_properties(**{'text-align': 'left'})
# df_operator_run_order.head(20)

In [85]:
# df_operator_run_order.info()

# Peak memory usage

In [86]:
models_peak_memory_path = models_dir.joinpath(model_name, f"{model_name}_peak_memory")
model_peak_memory_schedule = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "execution_schedule_info.csv")
peak_memory_tensor_details = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "tensor_details.csv")
models_peak_memory_path

PosixPath('/mnt/c/tiny_mlc/tiny_cnn/models/mobilenetv2_0.25_96_c3_o2_t5l512.MV1/mobilenetv2_0.25_96_c3_o2_t5l512.MV1_peak_memory')

In [87]:
models_tflite_opt_path

PosixPath('/mnt/c/tiny_mlc/tiny_cnn/models/mobilenetv2_0.25_96_c3_o2_t5l512.MV1/mobilenetv2_0.25_96_c3_o2_t5l512.MV1_INT8.tflite')

In [88]:
tflite_tool_path = Path.cwd().parent.joinpath("tflite-tools", "tflite_tools.py")
tflite_tool_path

PosixPath('/mnt/c/tiny_mlc/tflite-tools/tflite_tools.py')

In [89]:
#! python i:\tinyml\tflite-tools\tflite_tools.py -i $models_tflite_opt_path --csv $models_peak_memory_path
#! python c:\tiny_mlc\tflite-tools\tflite_tools.py -i $models_tflite_opt_path --csv $models_peak_memory_path
! python $tflite_tool_path -i $models_tflite_opt_path --csv $models_peak_memory_path

Writing model analysis to /mnt/c/tiny_mlc/tiny_cnn/models/mobilenetv2_0.25_96_c3_o2_t5l512.MV1/mobilenetv2_0.25_96_c3_o2_t5l512.MV1_peak_memory in CSV format


## redundant code

In [90]:
# def format_shape(x):
#     x = str(x)
#     x = x.strip()
#     split_strings = x.split()
#     joined_string = ", ".join(split_strings)
#     #print(joined_string)
#     joined_string = "(" + joined_string +")"
#     return joined_string     

In [91]:
# def split_operator_col(x):
#     x = str(x).strip("()")
#     split_strings = x.split(";")
#     #print(split_strings)
#     return split_strings[-1].strip()

In [92]:
# def get_middle_part(x):
#     x = str(x).strip("()")
#     split_strings = x.split("/")
#     if len(split_strings) == 3:
#         return split_strings[1]
#     elif len(split_strings) ==4:
#         return split_strings[1]
#     else:
#         return x

In [93]:
# def clean_tensor_memory_df(df):
    
#     df["name"] = df["Name"].apply(split_operator_col)
#     df["Size"] = df["Size"].map(int)
#     df["Name"] = df["Name"].apply(get_middle_part)
    
#     df["Shape"].fillna(0, inplace=True)
#     df["Shape"] = df["Shape"].apply(format_shape)

#     # reorder columns
#     df.insert(1, "name", df.pop("name"))
#     df.insert(4, "Name", df.pop("Name"))

#     # rename columns
#     df.columns = ["id", "name", "shape", "size_b", "name_long"]

#     return df

In [94]:
# def get_tensor_details_df(filepath):
#     print(f"Reading in {filepath}")
#     df = pd.read_csv(filepath)
#     print("Cleaning up the dataframe.")
#     df = clean_tensor_memory_df(df)
#     df = df.reset_index()
#     return df

In [95]:
# 'df = pd.read_csv(peak_memory_tensor_details)
# df'

## hier geht es weiter


In [96]:
tensor_details_df = get_tensor_details_df(peak_memory_tensor_details)
tensor_details_df.head()

Reading in /mnt/c/tiny_mlc/tiny_cnn/models/mobilenetv2_0.25_96_c3_o2_t5l512.MV1/mobilenetv2_0.25_96_c3_o2_t5l512.MV1_peak_memory/tensor_details.csv
Cleaning up the dataframe.


,index,id,layer_name,shape,size_b,name_long
0,0,0,serving_default_input_1:0,"(1, 96, 96, 3)",27648,serving_default_input_1:0
1,1,107,tfl.quantize,"(1, 96, 96, 3)",27648,tfl.quantize
2,2,108,mobilenetv2/conv2d_84/Conv2D,"(1, 48, 48, 8)",18432,mobilenetv2/conv1_relu/Relu6;mobilenetv2/conv1_bn/FusedBatchNormV3;mobilenetv2/block_0_dw_bn/FusedBatchNormV3/ReadVariableOp;mobilenetv2/conv2d_84/Conv2D
3,3,109,mobilenetv2/block_5_compress_bn/FusedBatchNormV3,"(1, 48, 48, 8)",18432,mobilenetv2/block_0_dw_relu/Relu6;mobilenetv2/block_0_dw_bn/FusedBatchNormV3;mobilenetv2/block_0_dw_bn/FusedBatchNormV3/ReadVariableOp;mobilenetv2/block_0_dw_conv/depthwise;mobilenetv2/block_5_compress_bn/FusedBatchNormV3
4,4,110,mobilenetv2/block_0_compress/Conv2D,"(1, 48, 48, 4)",9216,mobilenetv2/block_0_compress_bn/FusedBatchNormV3;mobilenetv2/block_0_compress_bn/FusedBatchNormV3/ReadVariableOp;mobilenetv2/block_0_compress/Conv2D


## Peak memory df

In [97]:
# def clean_working_set(x):
#     split_strings = x.split()
#     if len(split_strings) == 2:
#         return (int(split_strings[0]),int(split_strings[1]))
#     elif len(split_strings) == 3:
#         return (int(split_strings[0]),int(split_strings[1]),int(split_strings[2]))
#     else:
#         return x

In [98]:
# def clean_peak_memory_df(df):
    
#     df["Name"] = df["Operator"].apply(split_operator_col)
#     df["Memory use"] = df["Memory use"].map(int)
#     df["Name"] = df["Name"].apply(get_middle_part)
#     df["Working set"] = df["Working set"].apply(clean_working_set)

#     # reorder columns
#     df.insert(0, "Name", df.pop("Name"))
#     df.insert(3, "Operator", df.pop("Operator"))

#     # rename columns
#     df.columns = ["name", "tensor_IDs", "RAM_b", "operator"]

#     return df

In [99]:
# def get_peak_memory_df(filepath):
#     print(f"Reading in {filepath}")
#     df = pd.read_csv(filepath)
#     print("Cleaning up the dataframe.")
#     df = clean_peak_memory_df(df)
#     df = df.reset_index()
#     return df

In [100]:
peak_memory_df = get_peak_memory_df(model_peak_memory_schedule)
peak_memory_df.head()

Reading in /mnt/c/tiny_mlc/tiny_cnn/models/mobilenetv2_0.25_96_c3_o2_t5l512.MV1/mobilenetv2_0.25_96_c3_o2_t5l512.MV1_peak_memory/execution_schedule_info.csv
Cleaning up the dataframe.


,index,layer_name,tensor_IDs,RAM_b,operator
0,0,tfl.quantize,"(0, 107)",55296,tfl.quantize
1,1,conv2d_84,"(107, 108)",46080,mobilenetv2/conv1_relu/Relu6;mobilenetv2/conv1_bn/FusedBatchNormV3;mobilenetv2/block_0_dw_bn/FusedBatchNormV3/ReadVariableOp;mobilenetv2/conv2d_84/Conv2D
2,2,block_5_compress_bn,"(108, 109)",36864,mobilenetv2/block_0_dw_relu/Relu6;mobilenetv2/block_0_dw_bn/FusedBatchNormV3;mobilenetv2/block_0_dw_bn/FusedBatchNormV3/ReadVariableOp;mobilenetv2/block_0_dw_conv/depthwise;mobilenetv2/block_5_compress_bn/FusedBatchNormV3
3,3,block_0_compress,"(109, 110)",27648,mobilenetv2/block_0_compress_bn/FusedBatchNormV3;mobilenetv2/block_0_compress_bn/FusedBatchNormV3/ReadVariableOp;mobilenetv2/block_0_compress/Conv2D
4,4,block_1_expand,"(110, 111)",55296,mobilenetv2/block_1_expand_relu/Relu6;mobilenetv2/block_1_expand_bn/FusedBatchNormV3;mobilenetv2/block_1_dw_bn/FusedBatchNormV3/ReadVariableOp;mobilenetv2/block_1_expand/Conv2D


In [101]:
peak_memory_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       68 non-null     int64 
 1   layer_name  68 non-null     object
 2   tensor_IDs  68 non-null     object
 3   RAM_b       68 non-null     int64 
 4   operator    68 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.8+ KB


# Logging to wandb

In [102]:
# Generate run ids



# PROJECT = model_name.split("_")[0]

PROJECT = "model_DB"

run = wandb.init(
        # Set the project where this run will be logged
        project=PROJECT, 
        name = model_name,
        id = id, 
        resume="allow",
        sync_tensorboard=True
        )
# Specify the configuration variables
config = wandb.config
config.architecture = model_name

run.log(model_stats)
run.log(prof_stats)

# # Create a table
operator_top_by_time_table = wandb.Table(dataframe=operator_by_comp_time_df)
# operator_top_by_time_artifact = wandb.Artifact(name=f'{model_name}_operator_top_by_time', type='dataframe')
# operator_top_by_time_artifact.add(operator_top_by_time_table, "Profiling operators - top by time")
# run.log_artifact(operator_top_by_time_artifact)

summary_by_node_type_table = wandb.Table(dataframe=summary_node_type_df)
# summary_by_node_type_artifact = wandb.Artifact(name=f'{model_name}_summary_by_node_type', type='dataframe')
# summary_by_node_type_artifact.add(summary_by_node_type_table, "Node type summary")
# run.log_artifact(summary_by_node_type_artifact)

operator_run_order_table = wandb.Table(dataframe=operator_run_order_df)
# operator_run_order_artifact = wandb.Artifact(name=f'{model_name}_operator_run_order', type='dataframe')
# operator_run_order_artifact.add(operator_run_order_table, "Run order")
# run.log_artifact(operator_run_order_artifact)


run.log({"Profiling operators - top by time": operator_top_by_time_table})
run.log({"Node type summary" : summary_by_node_type_table})
run.log({"Run order" : operator_run_order_table})
# run.log({"Chart_table": table})

# table = wandb.Table(columns = ["plotly_figure"])

# # Create path for Plotly figure
# path_to_plotly_html = "./plotly_figure.html"

# # Write Plotly figure to HTML
# fig.write_html(path_to_plotly_html, auto_play = False) # Setting auto_play to False prevents animated Plotly charts from playing in the table automatically

# # Add Plotly figure as HTML file into Table
# table.add_data(wandb.Html(path_to_plotly_html))

# # Log Table
# run.log({"Chart_table": table})

#wandb.log(model_stats)

# #wandb.log({"Peak activations chart": fig})

wandb.finish()

allocate_tensors_ms_%,▁▁
allocate_tensors_ms_avg,▁▁
allocate_tensors_ms_first,▁▁
first_inference_us,▁▁
inference_avg_us,▁▁
init_us,▁▁
initialization_ms,▁▁
model_size_MB,▁▁
modify_graph_with_delegate_mem_KB,▁▁
modify_graph_with_delegate_ms_%,▁▁
modify_graph_with_delegate_ms_avg,▁▁
